In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_validate 
import random 
from sklearn.model_selection import KFold  
from sklearn.ensemble import RandomForestClassifier

load dataset from seaborn

In [2]:
df = sns.load_dataset('penguins')
df.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female


In [3]:
df.dropna(inplace = True) 
df.duplicated().sum()

0

In [9]:
X = df[['bill_length_mm' ,	'bill_depth_mm'	, 'flipper_length_mm']]  
y = df['species']  
max_feat = range(1,100,1)  
max_feat = [x/100 for x in max_feat] 
init = 0 
tmax = 300 
ctemp = 0 
cv = KFold(n_splits=5, shuffle=True) 
init=0   
max_depths = [x for x in range(1,50)] 
n_estimator = [x for x in range(1,100)] 
init_max_feat = 0
init_max_depth = 0 
init_n_estimators = 0 
max = 0
model = RandomForestClassifier(max_features=max_feat[init_max_feat],
                                   random_state=1, 
                                   n_estimators= n_estimator[init_n_estimators],      #model initialization
                                   max_depth = max_depths[init_max_depth])  
while(tmax > 0)  :   
    next_init = {'next_init_max_depth' : np.random.randint(0,len(max_depths)), 
                 'next_init_n_estimators' : np.random.randint(0, len(n_estimator)),  #random every params for next iterations
                 'next_init_max_feat' : np.random.randint(0, len(max_feat))}  
   
    next_model = RandomForestClassifier(max_features=max_feat[next_init['next_init_max_feat']],
                                         random_state=1, 
                                         n_estimators = n_estimator[next_init['next_init_n_estimators']], #initiate next model with randomly generated value.
                                         max_depth = max_depths[next_init['next_init_max_depth']]) 
    c_eval = cross_validate(estimator = model , X = X,y=y, scoring='accuracy', return_train_score=True, cv=cv) 
    n_eval = cross_validate(estimator = next_model , X = X,y=y, scoring='accuracy', return_train_score=True, cv=cv) 
    current_score = np.mean(c_eval['test_score']) 
    next_score = np.mean(n_eval['test_score']) 

    if (tmax ==300) : 
      max = current_score  #max initialization 
      best_model = model
      temperature = tmax

    else : 
      if (current_score > max) : 
         max = current_score     #climb. 
         best_temp = tmax 
         best_model = model
         temperature = tmax 
         tmax = 300 # restart checkpoint.

    score_diff = next_score - current_score 
    if (score_diff > 0) : 
       model = next_model  

    if (tmax%50==0) : 
       print('current ') 
       print('max_depth : ', model.get_params()['max_depth']) 
       print('n_estimator : ', model.get_params()['n_estimators'])
       print('max_feat : ', model.get_params()['max_features'])
       print('temperature : ', tmax) 
       print('current_score : ', current_score) 
       
       print('='*90) 

       print('best ') 
       print('max_depth : ', best_model.get_params()['max_depth']) 
       print('n_estimator : ', best_model.get_params()['n_estimators'])
       print('max_feat : ', best_model.get_params()['max_features'])
       print('temperature : ', temperature) 
       print('max_score : ', max) 
       print('='*90)
    tmax = tmax - 1

    
    

current 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  300
current_score :  0.753459972862958
best 
max_depth :  1
n_estimator :  1
max_feat :  0.01
temperature :  300
max_score :  0.753459972862958
current 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  300
current_score :  0.9700587969244685
best 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  299
max_score :  0.9700587969244685
current 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  300
current_score :  0.9759384893713252
best 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  299
max_score :  0.9759384893713252
current 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  300
current_score :  0.9760289461781998
best 
max_depth :  10
n_estimator :  81
max_feat :  0.35
temperature :  299
max_score :  0.9760289461781998
current 
max_depth :  34
n_estimator :  80
max_feat :  0.11
temperature :  300
current_score :  0.97901402080506

In [10]:
best_model

RandomForestClassifier(max_depth=41, max_features=0.49, n_estimators=23,
                       random_state=1)

Trying feature selection

In [235]:
def find_feat() : 
    feat = ['bill_length_mm','bill_depth_mm', 'flipper_length_mm', 'body_mass_g'] 
    leng = np.random.randint(1,len(feat)+1) 
    randfeat=[]
    for f in range(0,leng) : 
        randfeat.append(feat[np.random.randint(0,len(feat))]) 
    return list(np.unique(randfeat))
    

In [258]:
y = df.species 
temperature = 300  
cv = KFold(n_splits=5, shuffle=True) 
max_score = 0 
best_feat = []
while(temperature > 0) : 
    val_1 = find_feat() 
    val_2 = find_feat()
    #X_1 = df[val_1] 
    #X_2 = df[val_2] 
    #print(val_1,' ', val_2)
    current_result = cross_validate(estimator=best_model, X=X_1, y=y,scoring = 'accuracy') 
    current_score = np.mean(current_result['test_score'])
    next_result =   cross_validate(estimator=best_model, X=X_2, y=y, scoring = 'accuracy') 
    next_score = np.mean(next_result['test_score'])
    #if temperature == 300 : 
        #best_feat = val_1 
    if (current_score> max_score) : 
        max_score = current_score
        best_feat = val_1
        temperature = 300
    elif (next_score)> max_score :  
        max_score = next_score 
        best_feat = val_2
        temperature = 300 
    #print(np.mean(next_result['test_score'])) 
    #print(np.mean(current_result['test_score'])) 
    temperature = temperature - 1 
    print('feat_1, ', val_1) 
    print('score_1 ',current_score) 
    print('feat_2 ', val_2) 
    print('score_next', next_score)
    print('best_feat' , best_feat) 
    print('max_score', max_score) 
    print('temperature : ', temperature) 

feat_1,  ['bill_depth_mm', 'body_mass_g', 'flipper_length_mm']
score_1  0.7717322478516507
feat_2  ['bill_depth_mm', 'bill_length_mm', 'body_mass_g']
score_next 0.9729081863410223
best_feat ['bill_depth_mm', 'body_mass_g', 'flipper_length_mm']
max_score 0.7717322478516507
temperature :  299
feat_1,  ['bill_length_mm']
score_1  0.7717322478516507
feat_2  ['bill_depth_mm', 'flipper_length_mm']
score_next 0.9729081863410223
best_feat ['bill_depth_mm', 'flipper_length_mm']
max_score 0.9729081863410223
temperature :  299
feat_1,  ['bill_depth_mm', 'flipper_length_mm']
score_1  0.7717322478516507
feat_2  ['body_mass_g']
score_next 0.9729081863410223
best_feat ['bill_depth_mm', 'flipper_length_mm']
max_score 0.9729081863410223
temperature :  298
feat_1,  ['body_mass_g']
score_1  0.7717322478516507
feat_2  ['bill_length_mm', 'body_mass_g']
score_next 0.9729081863410223
best_feat ['bill_depth_mm', 'flipper_length_mm']
max_score 0.9729081863410223
temperature :  297
feat_1,  ['bill_depth_mm', 'b

In [260]:
best_feat

['bill_depth_mm', 'flipper_length_mm']